In [1]:
import pandas as pd

In [2]:
url_train = "https://github.com/sonder-art/automl_p24/raw/main/codigo/ml_practico/data_train.parquet"
data_train = pd.read_parquet(url_train)
data_train.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,charged_off
62394,29194239,NaN,3600.0,3600.0,3600.0,36 months,14.49,123.90,C,C4,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
2333,37620255,NaN,20000.0,20000.0,20000.0,36 months,12.99,673.79,C,C2,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
91099,24825804,NaN,10000.0,10000.0,10000.0,60 months,9.17,208.41,B,B1,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
31904,33192059,NaN,1500.0,1500.0,1500.0,36 months,12.39,50.11,C,C1,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
58152,29605392,NaN,21000.0,21000.0,21000.0,36 months,9.17,669.46,B,B1,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0


In [3]:
url_test = "https://github.com/sonder-art/automl_p24/raw/main/codigo/ml_practico/data_test.parquet"
data_test = pd.read_parquet(url_test)
data_test.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,charged_off
92063,24705541,NaN,10000.0,10000.0,10000.0,36 months,11.67,330.57,B,B4,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
88089,25217087,NaN,20000.0,20000.0,20000.0,36 months,12.99,673.79,C,C1,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
63335,29233877,NaN,20000.0,20000.0,20000.0,60 months,16.29,489.45,D,D2,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,1
63184,28683058,NaN,8000.0,8000.0,8000.0,36 months,10.99,261.88,B,B3,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0
96477,23905835,NaN,28000.0,28000.0,28000.0,36 months,6.03,852.20,A,A1,...,NaN,Cash,N,None,None,None,NaN,NaN,NaN,0


# EDA

## Feature Selection

Dada la cantidad de variables, en esta ocasion reduciremos el numero de variables antes de continuar.

Nuestra base de datos tiene 150 variables y 100 mil observaciones. Seria conveniente reducir la dimensionalidad. Sugerimos 4 criterios de **eliminacion**:
+ Baja correlacion con la variable objetivo
+ Alta correlacion entre ellas
+ Muchos Missings/NAs
+ Conocimiento del fenomeno/negocio 

### Baja correlacion con el objetivo

Una baja correlacion con el objetivo implica que explica poco.

In [4]:
data_train['charged_off']

62394    0
2333     0
91099    0
31904    0
58152    0
        ..
2005     1
64585    0
86108    0
6733     0
46758    0
Name: charged_off, Length: 68910, dtype: uint8

In [5]:
correlacion = data_train.corr(numeric_only=True)
correlacion_y = abs(correlacion['charged_off'])

In [6]:
correlacion_y.sort_values(ascending=False)

charged_off                                   1.000000
last_fico_range_high                          0.661246
last_fico_range_low                           0.581128
recoveries                                    0.517982
collection_recovery_fee                       0.497914
                                                ...   
hardship_length                                    NaN
orig_projected_additional_accrued_interest         NaN
settlement_amount                                  NaN
settlement_percentage                              NaN
settlement_term                                    NaN
Name: charged_off, Length: 117, dtype: float64

Los valores con NaNs es por que no tienen observaciones, las columnas estan llenas de missings.

In [7]:
correlacion_y.describe()

count    77.000000
mean      0.105295
std       0.172727
min       0.000684
25%       0.010804
50%       0.048290
75%       0.086554
max       1.000000
Name: charged_off, dtype: float64

In [8]:
drop_corr = sorted(list(correlacion_y[correlacion_y < 0.03].index))
print(f'Numero de columnas a quitar {len(drop_corr)} \n',drop_corr)

Numero de columnas a quitar 28 
 ['acc_now_delinq', 'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'delinq_2yrs', 'delinq_amnt', 'id', 'mo_sin_old_il_acct', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_rev_accts', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'pct_tl_nvr_dlq', 'pub_rec', 'pub_rec_bankruptcies', 'revol_bal', 'tax_liens', 'tot_coll_amt', 'total_acc', 'total_bal_ex_mort', 'total_il_high_credit_limit']


In [9]:
data_train.drop(labels=drop_corr, axis=1, inplace=True)
data_train.shape

(68910, 122)

In [10]:
data_test.drop(labels=drop_corr, axis=1, inplace=True)
data_test.shape

(17228, 122)

### Eliminacion por Missings

La cota de .3 es totalmente arbitraria, una buena regla de dedo para este tipo de datos es entre .25 a .5, pero puede variar mucho.

In [11]:
missing_per = data_train.isnull().mean().sort_values(ascending=False)
missing_per.describe()

count    122.000000
mean       0.442512
std        0.494833
min        0.000000
25%        0.000000
50%        0.005660
75%        1.000000
max        1.000000
dtype: float64

In [12]:
drop_miss_tr = sorted(list(missing_per[missing_per > 0.3].index))
print(f'Numero de columnas a quitar {len(drop_miss_tr)} \n',drop_miss_tr)

Numero de columnas a quitar 54 
 ['all_util', 'annual_inc_joint', 'debt_settlement_flag_date', 'deferral_term', 'desc', 'dti_joint', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type', 'il_util', 'inq_fi', 'inq_last_12m', 'max_bal_bc', 'member_id', 'mths_since_last_record', 'mths_since_rcnt_il', 'next_pymnt_d', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m', 'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 'revol_bal_joint', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_mths_since_last_major_derog', 'sec_app_num_rev_accts', 'sec_app_open_acc', 'sec_app_open_act_il', 'sec_a

In [13]:
missing_per = data_test.isnull().mean().sort_values(ascending=False)
missing_per.describe()

count    122.000000
mean       0.442564
std        0.494750
min        0.000000
25%        0.000000
50%        0.006008
75%        1.000000
max        1.000000
dtype: float64

In [14]:
drop_miss_ts = sorted(list(missing_per[missing_per > 0.3].index))
print(f'Numero de columnas a quitar {len(drop_miss_ts)} \n',drop_miss_ts)

Numero de columnas a quitar 54 
 ['all_util', 'annual_inc_joint', 'debt_settlement_flag_date', 'deferral_term', 'desc', 'dti_joint', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type', 'il_util', 'inq_fi', 'inq_last_12m', 'max_bal_bc', 'member_id', 'mths_since_last_record', 'mths_since_rcnt_il', 'next_pymnt_d', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m', 'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 'revol_bal_joint', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_mths_since_last_major_derog', 'sec_app_num_rev_accts', 'sec_app_open_acc', 'sec_app_open_act_il', 'sec_a

In [15]:
drop_miss = drop_miss_ts + drop_miss_tr
data_test.drop(labels=drop_miss, axis=1, inplace=True)
data_train.drop(labels=drop_miss, axis=1, inplace=True)


In [16]:
data_test.shape

(17228, 68)

In [17]:
data_train.shape

(68910, 68)

### Conocimiento o Intuicion

Estas son algunas de las variables sugeridas por la comundiad de kaggle, sin embargo. Combiene que las revisemos nosotros manualmente.   
Tambien devemos enfocarlos en eliminar leakages, y variables que no conoceremos *a priori*. Para ello tambien nos guiamos del siguiente documento:  [Credit Risk Analysis in Peer to Peer Lending Data set: Lending Club Club](https://digitalcommons.bard.edu/cgi/viewcontent.cgi?article=1299&context=senproj_s2019). Que es muy parecido al ejercicio que estamos realizando.

In [18]:
print(sorted(data_train.columns))

['acc_open_past_24mths', 'addr_state', 'annual_inc', 'application_type', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'charged_off', 'collection_recovery_fee', 'debt_settlement_flag', 'disbursement_method', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'initial_list_status', 'inq_last_6mths', 'installment', 'int_rate', 'issue_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'loan_amnt', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_op_rev_tl', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_op_past_12m', 'open_acc', 'out_prncp', 'out_prncp_inv', 'percent_bc_gt_75', 'policy_code', 'purpose', 'pymnt_plan', 'recoveries', 'revol_util', 'sub_grade', 'term', 'title', 'tot_cur_bal', 'tot_hi_cred_li

In [19]:
elegidas = ['charged_off','funded_amnt','addr_state', 'annual_inc',
            'application_type', 'dti', 'earliest_cr_line', 'emp_length', 
            'emp_title', 'fico_range_high', 'fico_range_low', 'grade',
            'home_ownership', 'initial_list_status', 'installment', 
            'int_rate', 'loan_amnt', 'loan_status', 'mort_acc', 
            'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 
            'revol_bal', 'revol_util', 'sub_grade', 'term', 'title', 
            'total_acc', 'verification_status', 'zip_code',
            'last_pymnt_amnt','num_actv_rev_tl', 'mo_sin_rcnt_rev_tl_op',
            'mo_sin_old_rev_tl_op',"bc_util","bc_open_to_buy","avg_cur_bal",
            "acc_open_past_24mths" ]
len(elegidas)

39

In [20]:
drop_no_intuitivas = [col for col in data_train.columns if col not in elegidas]
data_train.drop(labels=drop_no_intuitivas , axis=1, inplace=True)
data_test.drop(labels=drop_no_intuitivas , axis=1, inplace=True)

In [21]:
data_train.shape

(68910, 34)

In [22]:
data_test.shape

(17228, 34)

In [23]:
data_train.to_parquet('data_train_fs.parquet')
data_test.to_parquet('data_test_fs.parquet')